In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
# from ray import train, tune
# from ray.tune.search.optuna import OptunaSearch
# from ray.tune.s|chedulers import ASHAScheduler
# from ray.tune.search import ConcurrencyLimiter

/opt/miniconda3/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


In [55]:
fn_train_x = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/train_x.parquet'
fn_train_y = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/train_y.parquet'
fn_train_id = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train/test_id.parquet'
# dir_hyperparameters = '/Volumes/Extreme SSD/rematch_eia_ferc1_docker/working_data/model_a/train'

Demonstrate model goodness-of-fit statistics

NB- each time raytune runs, it uses a different subset of the data- so the hyperparameter search (and its resulting AUCs and binary logloss metrics) were _not_ subject to leakage

In [57]:
X = pd.read_parquet(fn_train_x)
Y = pd.read_parquet(fn_train_y)
ID = pd.read_parquet(fn_train_id)

In [60]:
ID.record_id_ferc1.value_counts()

record_id_ferc1
f1_steam_2005_12_210_1_3    467
f1_hydro_2012_12_57_1_5     464
f1_steam_2016_12_99_0_4     464
f1_hydro_2019_12_186_0_2    463
f1_steam_2017_12_70_0_1     460
                           ... 
f1_steam_2014_12_57_1_5     338
f1_hydro_2005_12_134_1_2    337
f1_steam_2004_12_6_0_1        1
f1_steam_2004_12_79_1_5       1
f1_steam_2004_12_44_5_1       1
Name: count, Length: 8024, dtype: int64

In [51]:
num_folds = 5
fold_array = np.random.choice(a=range(num_folds), size=X.shape[0], replace=True)
params = {'learning_rate':0.0160, 'min_data_in_leaf':163, 'num_iterations':450}

In [52]:
fold = 0
is_train_mask = fold_array != fold

train_set = lgb.Dataset(X.loc[is_train_mask], Y.loc[is_train_mask])
# test_set = lgb.Dataset(X.loc[~is_train_mask], Y.loc[~is_train_mask])

mod_fit = lgb.train(train_set=train_set, params=params)

/opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning(f"Found `{alias}` in params. Will use it instead of argument")


[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.627447 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 2895
[LightGBM] [Info] Number of data points in the train set: 10270983, number of used features: 57
[LightGBM] [Info] Start training from score 0.000500


In [54]:
mod_fit.predict(X.loc[~is_train_mask])

array([ 3.81124201e-05,  2.91890802e-05, -7.90502318e-05, ...,
       -1.45751114e-05,  2.61739336e-04,  2.39382389e-04])

In [13]:
def fit_mod(space):

    # ELT
    X = pd.read_parquet(fn_train_x)
    Y = pd.read_parquet(fn_train_y)
    # X = pd.read_parquet('/Volumes/Extreme SSD/rematch1_predictor/full_training_data/train_x.parquet')
    # Y = pd.read_parquet('/Volumes/Extreme SSD/rematch1_predictor/full_training_data/full_data_y.parquet')
    
    size_of_train_set = round(0.8 * X.shape[0])
    rows_for_train_set = np.random.choice(a=X.index, size=size_of_train_set, replace=False)
    rows_for_val_set = np.setdiff1d(X.index, rows_for_train_set)
    
    train_set = lgb.Dataset(X.loc[rows_for_train_set], Y.loc[rows_for_train_set])
    val_set = lgb.Dataset(X.loc[rows_for_val_set], Y.loc[rows_for_val_set])

    # Model
    gbm = lgb.train(
        space,
        train_set,
        valid_sets=[val_set],
    )
    binary_logloss = gbm.best_score['valid_0']['binary_logloss']
    auc = gbm.best_score['valid_0']['auc']
    train.report(
        {
            "binary_logloss": binary_logloss,
            "auc": auc
        }
    )

In [1]:
space = {
    'num_iterations': tune.randint(1, 500),
    # 'num_rounds': tune.randint(1, 500),
    'learning_rate': tune.uniform(0.0001, 1),
    'min_data_in_leaf': tune.randint(1, 200),
    'objective':'binary', 
    # 'early_stopping_round':2,
    'metrics':['binary_logloss', 'auc']
    }

NameError: name 'tune' is not defined

In [17]:
# asha_scheduler = ASHAScheduler(
#     time_attr='training_iteration',
#     metric='binary_logloss',
#     mode='min',
#     max_t=1000,
#     grace_period=50,
#     reduction_factor=3,
#     brackets=1,
# )

search_alg = OptunaSearch(metric="binary_logloss", mode="min")
search_alg = ConcurrencyLimiter(search_alg, max_concurrent=1)

In [18]:
tuner = tune.Tuner(
    fit_mod,
    tune_config=tune.TuneConfig(
        # scheduler=asha_scheduler,
        search_alg=search_alg,
        num_samples=1000
    ),
    param_space=space,
    run_config=train.RunConfig(
        storage_path=dir_hyperparameters, 
        name="gb_ray_tune"
    )
)
results = tuner.fit()

(pid=28623) /opt/miniconda3/lib/python3.12/site-packages/dask/dataframe/__init__.py:31: FutureWarning: 
(pid=28623) Dask dataframe query planning is disabled because dask-expr is not installed.
(pid=28623) 
(pid=28623) You can install it with `pip install dask[dataframe]` or `conda install dask`.
(pid=28623) This will raise in a future version.
(pid=28623) 
(pid=28623)   warnings.warn(msg, FutureWarning)
(fit_mod pid=28623) /opt/miniconda3/lib/python3.12/site-packages/lightgbm/engine.py:204: UserWarning: Found `num_rounds` in params. Will use it instead of argument
(fit_mod pid=28623)   _log_warning(f"Found `{alias}` in params. Will use it instead of argument")
2024-12-17 20:25:39,561	WARNING tune.py:219 -- Stop signal received (e.g. via SIGINT/Ctrl+C), ending Ray Tune run. This will try to checkpoint the experiment state one last time. Press CTRL+C (or send SIGINT/SIGKILL/SIGTERM) to skip. 
2024-12-17 20:25:39,591	INFO tune.py:1009 -- Wrote the latest version of all result files and e

In [ ]:
# experiment_path = "/Users/andrewbartnof/Documents/rmi/rematch_ferc_eia1/clean_data/model_full_gradient_boost/ray_tune/gb_ray_tune"
# restored_tuner = tune.Tuner.restore(experiment_path, trainable=fit_mod)

In [ ]:
# fn_results = '/Users/andrewbartnof/Documents/rmi/rematch_ferc_eia1/clean_data/model_full_gradient_boost/ray_tune/ray_tune_dataframe.csv'
# restored_tuner.get_results().get_dataframe().to_csv(fn_results)

In [2]:
# !jupyter nbconvert --to script model_a_hyperparameter_search.ipynb

[NbConvertApp] Converting notebook model_a_hyperparameter_search.ipynb to script
[NbConvertApp] Writing 3141 bytes to model_a_hyperparameter_search.py
